In [1]:
from keras.models import Model
from keras.layers import Conv2D, Dense, Flatten,Dropout,Input
from keras.layers import MaxPooling2D,AveragePooling2D,Concatenate

In [2]:
def inception(x,n):
    conv5=Conv2D(x,n,(5,5),padding="same",activation="relu",kernel_initializer="he_uniform")
    conv3=Conv2D(x,n,(3,3),padding="same",activation="relu",kernel_initializer="he_uniform")
    conv1=Conv2D(x,n,(1,1),padding="same",activation="relu",kernel_initializer="he_uniform")
    merged=Concatenate(axis=3,name='inception_network')([conv1,conv3,conv5])
    x=MaxPooling2D(merged,pool_size=(3,3))
    return x

In [7]:
img_input=Input(shape=(512,512,3))
x=Conv2D(img_input,16,7,2,activation="relu",kernel_initializer="he_uniform")
x=MaxPooling2D(3,2,padding="valid")(x)
x=Conv2D(32,3,1,activation="relu",kernel_initializer="he_uniform")(x)
x=MaxPooling2D(3,2,padding="valid")(x)
x=inception(x,64)
x=inception(x,64)
x=MaxPooling2D(3,2,padding="valid")(x)
x=inception(x,128)
x=inception(x,128)
x=inception(x,128)
x=inception(x,128)
x=inception(x,128)
x=MaxPooling2D(3,2,padding="valid")(x)
x=inception(x,256)
x=inception(x,256)
x=AveragePooling2D(2,padding="same")(x)
x=Dropout(0.2)(x)
x=Flatten()(x)
x=Dense(units=128,activation='relu')(x)
x=Dense(units=5,activation='softmax')(x)

AttributeError: 'int' object has no attribute 'lower'

In [ ]:
model=Model(x,img_input)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# fittig the above model to the images
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.3,
                                   zoom_range = 0.3,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('data/trainA',
                                                 target_size = (512,512),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('data/trainB',
                                            target_size = (512,512),
                                            batch_size = 16,
                                            class_mode = 'categorical')

model.fit_generator(training_set, 
                         steps_per_epoch = 200,
                         epochs = 16,
                         validation_data = test_set,
                         validation_steps = 20)